In [1]:
!pip install transformers

     |████████████████████████████████| 1.8MB 6.6MB/s 
     |████████████████████████████████| 890kB 31.2MB/s 
     |████████████████████████████████| 3.2MB 40.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=6deea9a9d659726aeb5effc744ab1d570e30a52fcdd737fa88193d30bab72a83
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, Input
from transformers import TFDistilBertModel
import glob
from tensorflow.keras.utils import Progbar
from google.colab import drive
import os
from pathlib import Path
import json
import seaborn as sns
from matplotlib import pyplot as plt
import pickle as pkl
from reddit import load_tfrecord_triplet, split_dataset

In [3]:
drive.mount('/content/drive/')
%cd 'drive/My Drive/personality_reddit'

Mounted at /content/drive/
/content/drive/My Drive/personality_reddit


## Initialize TPU strategy

In [5]:
try:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
  print("Number of accelerators: ", strategy.num_replicas_in_sync)
except ValueError:
  print('TPU failed to initialize.')

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.109.133.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.109.133.186:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.109.133.186:8470']
Number of accelerators:  8


In [6]:
AUTO = tf.data.experimental.AUTOTUNE

## Define dataset parameters

In [ ]:
batch_size = 1 # >1 not feasible on Colab TPUs
global_batch_size = batch_size * strategy.num_replicas_in_sync
n_posts = 50
samples_in_toy_dataset = 10000

## Load and distribute dataset

In [8]:
from google.colab import auth
auth.authenticate_user()
!gcloud config set project thematic-cursor-254011

Updated property [core/project].


In [9]:
fre = 'gs://personality_reddit/triplet_nn1/*-of-999.tfrecord'
fnames = tf.io.gfile.glob(fre)

In [ ]:
ds = load_tfrecord_triplet_nn1(filenames=fnames, num_parallel_calls=AUTO)
ds = ds.map(lambda x: tf.reverse(x['input_ids'], [0])[:n_posts,:]) \
       .prefetch(AUTO) \
       .padded_batch(global_batch_size, 
                     padded_shapes=[n_posts, None],
                     drop_remainder=True)

In [12]:
ds_tuning, ds_train, ds_val, ds_test = split_dataset(ds, tuning=10000)
ds_tuning, ds_val, ds_test = (strategy.experimental_distribute_dataset(d) for d in [ds_tuning, ds_val, ds_test])

## Define model

In [ ]:
input_ids = Input(shape=(None, 512), dtype='int32', name='input_ids')

In [17]:
class CustomBertLayer(keras.Model):
  def __init__(self, model, name=None, trainable=True):
    super(CustomBertLayer, self).__init__(name=name)
    self.model = model
    self.trainable = trainable
    self.output_signature = tf.float32

  def call(self, input):
    def red_fn(x):
      enc = self.model(x).last_hidden_state[:,0,:]
      mask = tf.cast(tf.not_equal(tf.reduce_mean(x, axis=1), 0), 
                     tf.float32)
      masked = tf.multiply(enc, tf.expand_dims(mask, 1))
      return masked

    enc = tf.vectorized_map(lambda x: red_fn(x), elems=input)
    nonzero = tf.cast(tf.math.count_nonzero(tf.reduce_sum(input, axis=-1), axis=1), tf.float32)
    return enc, nonzero

In [18]:
def triplet_loss(margin):
  def loss_fn(enc, nonzero):

    # Get embeddings
    n_emb = enc[:,0,:]
    p_emb = enc[:,1,:]
    a_emb = enc[:,2:,:]
    
    # Compute average distance between anchor embeddings (ONLY WORKS for batch=1)
    r = tf.reduce_sum(a_emb[0,:,:] * a_emb[0,:,:], 1)
    mask = tf.abs(tf.cast(tf.equal(r,0), tf.float32) - 1.0)
    r = tf.reshape(r, [-1, 1])
    D = r - 2*tf.matmul(a_emb[0,:,:], 
                        tf.transpose(a_emb[0,:,:])) + tf.transpose(r)
    D = tf.transpose(D * mask) * mask
    D = tf.linalg.band_part(D,-1,0) # lower triangle
    nr_comb = tf.reduce_sum(tf.cast(tf.range(1,n_posts-1), tf.float32) * mask)
    mdist = tf.divide(tf.reduce_sum(D), nr_comb)

    # Compute distances
    a_emb = tf.divide(tf.reduce_sum(a_emb, axis=1), nonzero-2)
    d_pos = tf.reduce_sum(tf.square(a_emb - p_emb), 1)
    d_neg = tf.reduce_sum(tf.square(a_emb - n_emb), 1)

    # Compute loss
    loss_val = tf.maximum(0.0, margin + (d_pos - d_neg) )

    # Return
    loss_val = tf.reduce_mean(loss_val)
    d_pos = tf.reduce_mean(d_pos)
    d_neg = tf.reduce_mean(d_neg)
    
    return loss_val, d_pos, d_neg, mdist
  
  return loss_fn

## Training protocol

In [19]:
!pip install -q tf-models-official==2.3.0
from official.nlp.optimization import create_optimizer # creates AdamW optimizer used for BERT - tune?

     |████████████████████████████████| 849kB 6.4MB/s 
     |████████████████████████████████| 706kB 17.3MB/s 
     |████████████████████████████████| 1.2MB 25.6MB/s 
     |████████████████████████████████| 37.6MB 1.4MB/s 
     |████████████████████████████████| 174kB 50.4MB/s 
     |████████████████████████████████| 358kB 47.8MB/s 
     |████████████████████████████████| 102kB 8.9MB/s 


Define parameters for training

In [20]:
EPOCHS = 10
tot_train_steps = n_train * EPOCHS
warmup_steps = int(EPOCHS * n_train * 0.1 / global_batch_size)

Specify checkpoint to load (for epoch > 0)

In [21]:
lh_options = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")

Set up training functions

In [22]:
margins = [10 * 10**(x) for x in range(-3,3)]

In [24]:
def training_setup(lr, margin, load=False, batch_nr=27113):

  with strategy.scope():
    encoder_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
    encoder_model.trainable = True

    encs, nonzero = CustomBertLayer(encoder_model)(input_ids)
    model = keras.Model(input_ids, [encs, nonzero])
    model.trainable = encoder_model.trainable
    loss_obj = triplet_loss(margin)
    optimizer = create_optimizer(lr, 
                                 num_train_steps=tot_train_steps,
                                 num_warmup_steps=warmup_steps)
    opt_checkpoint = tf.train.Checkpoint(optimizer=optimizer)
    
    if load:
      latest = tf.train.latest_checkpoint(f'checkpoints_shuffle_02_10/triplet_loss_{margin}/epoch_{load}')
      print(f'Loading checkpoint at checkpoint/triplet_loss_{margin}/epoch_{load}...')
      model.load_weights(latest, options=lh_options)

      opt_wfile = f'optimizers_shuffle_02_10/triplet_loss_{margin}/epoch_{load}/batch_{batch_nr}-of-27113.pkl'
      opt_weights = pkl.load(file=open(opt_wfile, 'rb'))
      optimizer._create_all_weights(model.trainable_variables)
      optimizer.set_weights(opt_weights)

    def compute_loss(enc, nonzero):
      l, dp, dn, md = loss_obj(enc, nonzero)
      return l, dp, dn, md

    def compute_metric(loss, margin):
      m = tf.cast(tf.greater(margin, loss), tf.float32)
      return m

    def red_fn(x):
      return strategy.reduce(tf.distribute.ReduceOp.MEAN, x, axis=None)

    @tf.function
    def distributed_train_step(dataset_inputs):
      pr_losses, pr_metrics, pr_dp, pr_dn, mds = strategy.run(train_step, 
                                                         args=(dataset_inputs,))
      return red_fn(pr_losses), red_fn(pr_metrics), \
             pr_dp.values, pr_dn.values, mds.values

    @tf.function
    def distributed_test_step(dataset_inputs):
      pr_test_losses, pr_test_metrics = strategy.run(test_step, 
                                                    args=(dataset_inputs,))
      return red_fn(pr_test_losses), red_fn(pr_test_metrics)

    def train_step(inputs):
      with tf.GradientTape() as tape:
        enc, nz = model(inputs)
        loss, dp, dn, md = compute_loss(enc, nz)
        mtr = compute_metric(loss, margin)
      gradients = tape.gradient(loss, model.trainable_variables)
      optimizer.apply_gradients(zip(gradients, model.trainable_variables))
      return loss, mtr, dp, dn, md

    def test_step(inputs):
      test_enc, test_nz = model(inputs)
      test_loss = compute_loss(test_enc, test_nz)[0]
      test_mtr = compute_metric(test_loss, margin)
      return test_loss, test_mtr

    return model, loss_obj, optimizer, \
           distributed_train_step, distributed_test_step


Define logging functions

In [25]:
with strategy.scope():

  def log_weights(edir, b, model):
    edir.mkdir(exist_ok=True, parents=True)
    ckpt_path = edir /  f'batch_{b}-of-{n_train}'
    model.save_weights(filepath=ckpt_path, options=lh_options)
  
  def log_optimizer(edir, b, optimizer):
    edir.mkdir(exist_ok=True, parents=True)
    opt_wpath = edir / f'batch_{b}-of-{n_train}.pkl'
    pkl.dump(file=open(opt_wpath, 'wb'), 
             obj=optimizer.get_weights())

  def log_metrics(e, d, el, em, edps, edns, edas, logpath,
                  elt=None, emt=None, test=False):
    d[f'epoch_{e}']['losses'] = el
    d[f'epoch_{e}']['metrics'] = em
    if test:
      d[f'epoch_{e}']['test_losses'] = elt
      d[f'epoch_{e}']['test_metrics'] =  emt
    d[f'epoch_{e}']['dp'] = [float(e) for e in edps]
    d[f'epoch_{e}']['dn'] = [float(e) for e in edns]
    d[f'epoch_{e}']['da'] = [float(e) for e in edas]
    with open(logpath, 'w') as f:
      f.write(json.dumps(d))
    return d


Define training loop

In [26]:
def training_run(margin, dataset, dataset_val, 
                 ckptpath, mpath, optpath, 
                 epochs, examples, 
                 lr=2e-5, load=False, batch_nr=27113):

  with strategy.scope():

    model, loss_obj, optimizer, \
    distributed_train_step, distributed_test_step = training_setup(lr=lr, margin=margin, 
                                                                   load=load, batch_nr=batch_nr)
    print(optimizer.iterations)
    ckpt_dir = Path(f'./{ckptpath}/triplet_loss_{margin}')
    metrics_dir = Path(f'./{mpath}/triplet_loss_{margin}')
    optimizer_dir = Path(f'./{optpath}/triplet_loss_{margin}')
    metrics_dir.mkdir(exist_ok=True, parents=True)
    logpath = metrics_dir / 'log.json'

    if logpath.is_file():
      with open(logpath, 'r') as fh:
        mdict = json.load(fh)
    else:
      mdict = {}

    for epoch in range(int(load)+1,epochs):

      print(f'Epoch {epoch+1}/{epochs}')
      mdict[f'epoch_{epoch}'] = {}
      ckpt_epoch_dir = ckpt_dir / f'epoch_{epoch}'
      opt_epoch_dir = optimizer_dir / f'epoch_{epoch}'
      pb_i = Progbar(examples, stateful_metrics=['loss', 'correct'])

      epoch_losses, epoch_metrics, epoch_test_losses, epoch_test_metrics = [], [], [], []
      epoch_dps, epoch_dns, epoch_das = [], [], [] 
      batch = 0
      
      dataset_train = dataset.shuffle(examples)
      dataset_train = strategy.experimental_distribute_dataset(dataset_train)

      for x in dataset_train:
        batch += 1
        loss, correct, dp, dn, md = distributed_train_step(x)
        epoch_losses.append(float(loss.numpy()))
        epoch_metrics.append(float(correct.numpy()))
        epoch_das += [d.numpy() for d in md] 
        epoch_dps += [d.numpy() for d in dp]
        epoch_dns += [d.numpy() for d in dn] 
        pb_i.add(1, values=[('loss', loss), ('correct', correct)])
        if batch % 100 == 0:
          if (batch % 5000 == 0):
            log_weights(ckpt_epoch_dir, batch, model)
            log_optimizer(opt_epoch_dir, batch, optimizer)
          mdict = log_metrics(epoch, mdict, 
                              epoch_losses, epoch_metrics,
                              epoch_dps, epoch_dns, epoch_das, logpath)

      avg_train_loss = tf.reduce_sum(epoch_losses) / examples
      avg_train_mtr = tf.reduce_sum(epoch_metrics) / examples
      log_weights(ckpt_epoch_dir, batch, model)
      log_optimizer(opt_epoch_dir, batch, optimizer)

      for x in dataset_val:
        test_loss, test_correct = distributed_test_step(x)
        epoch_test_losses.append(float(test_loss.numpy()))
        epoch_test_metrics.append(float(test_correct.numpy()))

      avg_test_loss = tf.reduce_sum(epoch_test_losses) / n_val
      avg_test_mtr = tf.reduce_sum(epoch_test_metrics) / n_val
        
      print(f'Margin: {margin}, '
            f'Epoch: {epoch}, '
            f'Loss: {avg_train_loss}, ' 
            f'Metric: {avg_train_mtr} ,' 
            f'Val loss: {avg_test_loss}, ' 
            f'Val metric {avg_test_mtr}')
      
      mdict = log_metrics(epoch, mdict, 
                          epoch_losses, epoch_metrics,
                          epoch_dps, epoch_dns, epoch_das,
                          logpath, 
                          epoch_test_losses, epoch_test_metrics, 
                          test=True)

Run training loop for hyperparameter tuning with small dataset

In [ ]:
for m in margins[2:3]:
  training_run(float(m), ds_train, ds_val,
               'checkpoints_shuffle_02_10', 
               'metrics_shuffle_02_10', 
               'optimizers_shuffle_02_10',
                10, n_train, load='3')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_projector', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fd1d3ce42a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fd1d3ce42a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fd1d3ce42a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Loading checkpoint at checkpoint/triplet_loss_1.0/epoch_3...
TPUMirroredVariable:{
  0: <tf.Variable 'iter:0' shape=() dtype=int64, numpy=108452>,
  1: <tf.Variable 'iter/replica_1:0' shape=() dtype=int64, numpy=108452>,
  2: <tf.Variable 'iter/replica_2:0' shape=() dtype=int64, numpy=108452>,
  3: <tf.Variable 'iter/replica_3:0' shape=() dtype=int64, numpy=108452>,
  4: <tf.Variable 'iter/replica_4:0' shape=() dtype=int64, numpy=108452>,
  5: <tf.Variable 'iter/replica_5:0' shape=() dtype=int64, numpy=108452>,
  6: <tf.Variable 'iter/replica_6:0' shape=() dtype=int64, numpy=108452>,
  7: <tf.Variable 'iter/replica_7:0' shape=() dtype=int64, numpy=108452>
}
Epoch 5/10


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall:1", shape=(25600,), dtype=int32), values=Tensor("clip_by_global_norm/clip_by_global_norm/_0:0", shape=(25600, 768), dtype=float32), dense_shape=Tensor("PartitionedCall:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


27113/27113 [==============================] - 18295s 672ms/step - loss: 0.1983 - correct: 1.0000


Margin: 1.0, Epoch: 4, Loss: 0.03669251501560211, Metric: 0.9883036017417908 ,Val loss: 1.1744158267974854, Val metric 0.7145946025848389
Epoch 6/10
27113/27113 [==============================] - 18578s 684ms/step - loss: 0.0000e+00 - correct: 1.0000
Margin: 1.0, Epoch: 5, Loss: 0.02303762175142765, Metric: 0.9922592639923096 ,Val loss: 1.415094017982483, Val metric 0.7140459418296814
Epoch 7/10
  132/27113 [..............................] - ETA: 5:02:31 - loss: 0.0000e+00 - correct: 1.0000

KeyboardInterrupt: ignored

In [ ]:
# To dos:
# Note: cool embedding visualizer
# App for live monitoring of training

In [ ]:
#def triplet_loss(margin):
#  def loss_fn(enc, nonzero):
#    
    # Averaging distances
#    n_emb = enc[:,0,:]
#    p_emb = enc[:,1,:]
#    a_emb = enc[:,2:,:]
#    r = tf.reduce_sum(a_emb[0,:,:]*a_emb[0,:,:], 1)
#    r = tf.reshape(r, [-1, 1])
#    D = r - 2*tf.matmul(a_emb[0,:,:], tf.transpose(a_emb[0,:,:])) + tf.transpose(r)
#    mdist = tf.reduce_mean(D)
#    p_diff = p_emb - a_emb
#    n_diff = n_emb - a_emb
#    p_mask = tf.cast(tf.reduce_all((p_diff == p_emb), -1), tf.float32)
#    p_mask = tf.abs(p_mask - 1.0)
#    d_pos = tf.reduce_sum(tf.multiply(tf.reduce_sum(tf.square(p_diff), -1), p_mask), -1) / (nonzero-2)
#    d_neg = tf.reduce_sum(tf.multiply(tf.reduce_sum(tf.square(n_diff), -1), p_mask), -1) / (nonzero-2)
#    loss_val = tf.maximum(0.0, d_pos - d_neg + margin + 0.1*d_pos )
    
#    return tf.reduce_mean(loss_val), tf.reduce_mean(d_pos), tf.reduce_mean(d_neg), mdist
  
#  return loss_fn


#loss_val = tf.maximum(0.0, tf.math.log(d_pos/(mdist + 1e-8)) - tf.math.log(d_neg + 1e-8) - tf.math.log(1 - mdist + 1e-8) )